In [1]:
import numpy as np
import plotly.graph_objects as go
from sklearn.manifold import TSNE
from sklearn import metrics as sklearn_metrics

In [2]:
import sys
sys.path.append('./')

In [3]:
from datasets import Satimage2Dataset
from utils.metrics import *
from lof import LOF
import config

In [ ]:
knn = LOF(config)

knn.get_data()
knn.get_distances()
best_k_info = knn.get_best_k_by_AP()
print(f'best k: {best_k_info["best_k"]}')
l, dk2 = best_k_info['best_k_labels'], best_k_info['best_k_scores']

Files already downloaded and verified
k: 1, calculated AP: 0.10059999999999997
k: 2, calculated AP: 0.10059999999999997
k: 3, calculated AP: 0.10059999999999997
k: 4, calculated AP: 0.10046666666666663
k: 5, calculated AP: 0.10046666666666663
k: 6, calculated AP: 0.10046666666666663
k: 7, calculated AP: 0.10046666666666663
k: 8, calculated AP: 0.10046666666666663
k: 9, calculated AP: 0.10046666666666663
k: 10, calculated AP: 0.10046666666666663
k: 11, calculated AP: 0.10046666666666663
k: 12, calculated AP: 0.10046666666666663
k: 13, calculated AP: 0.10046666666666663
k: 14, calculated AP: 0.10046666666666663


c:\users\professional\appdata\local\programs\python\python39\scripts\torch_gpu\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


k: 15, calculated AP: 0.10046666666666663
k: 16, calculated AP: 0.10046666666666663
k: 17, calculated AP: 0.10046666666666663
k: 18, calculated AP: 0.10046666666666663
k: 19, calculated AP: 0.10046666666666663
k: 20, calculated AP: 0.10046666666666663
k: 21, calculated AP: 0.10046666666666663
k: 22, calculated AP: 0.10046666666666663
k: 23, calculated AP: 0.10046666666666663
k: 24, calculated AP: 0.10041904761904759
k: 25, calculated AP: 0.10041904761904759
k: 26, calculated AP: 0.10041904761904759
k: 27, calculated AP: 0.10041904761904759
k: 28, calculated AP: 0.10041904761904759
k: 29, calculated AP: 0.10041904761904759
k: 30, calculated AP: 0.10041904761904759
k: 31, calculated AP: 0.10041904761904759
k: 32, calculated AP: 0.10041904761904759
k: 33, calculated AP: 0.10041904761904759
k: 34, calculated AP: 0.10041904761904759
k: 35, calculated AP: 0.10041904761904759
k: 36, calculated AP: 0.10041904761904759
k: 37, calculated AP: 0.10041904761904759
k: 38, calculated AP: 0.1004190476

In [ ]:
p, r, t = precision_recall_curve(l, dk2)
ps, rs, ts = sklearn_metrics.precision_recall_curve(l, dk2)

In [ ]:
fig = go.Figure()

fig.add_trace(
        go.Scatter(
            line=dict(color="#00CED1", width=1),
            name='Sklearn',
            x=rs,
            y=ps))

fig.add_trace(
        go.Scatter(
            line=dict(color="#900C3F", width=1),
            name='Implemented',
            x=r,
            y=p))
fig.show()

In [ ]:
print(f'Implemented = {average_precision_score(p, r)}')
print(f'sklearn = {sklearn_metrics.average_precision_score(l, dk2)}')

In [ ]:
prj_v = TSNE(n_components=2).fit_transform(knn.vectors)

In [ ]:
f1_score_ = f1_score(p, r)
best_f1_score_idx = np.argmax(f1_score_)
best_f1_score = f1_score_[best_f1_score_idx]
print(f'best F1-score: {best_f1_score}')

best_thr = t[best_f1_score_idx]
fin_prediction = np.zeros(knn.dataset_len)
fin_prediction[knn.get_anomaly_scores(sorted_dists, k=best_k_info["best_k"]) > best_thr] = 1

In [ ]:
fig = go.Figure()

fig.add_trace(
        go.Scattergl(
            mode='markers',
            marker = dict(
            line = dict(
                width = 1,
                color = 'DarkSlateGrey')
            ),
            name='Normal',
            x=prj_v[fin_prediction == 0, 0],
            y=prj_v[fin_prediction == 0, 1]))

fig.add_trace(
        go.Scattergl(
            mode='markers',
            marker = dict(
            line = dict(
                width = 1,
                color = 'Crimson')
            ),
            name='Anomaly',
            x=prj_v[fin_prediction == 1, 0],
            y=prj_v[fin_prediction == 1, 1]))

fig.show()

In [ ]:
conf_matrix_for_best_thr = confusion_matrix(knn.labels, fin_prediction)

In [ ]:
import plotly.figure_factory as ff

z = conf_matrix_for_best_thr
z = z[::-1]

x = ['Normal', 'Anomaly']
y =  x[::-1].copy()

z_text = [[str(y) for y in x] for x in z]

fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  xaxis = dict(title='Predicted'),
                  yaxis = dict(title='Actual')
                 )

fig['data'][0]['showscale'] = True
fig.show()